In [1]:
# Initial imports.
import pandas as pd
import numpy as np
from path import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
# Data loading
df_raw = pd.read_csv("star_classification_raw.csv")
df_raw.head(10)

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
5,1.237680e+18,340.995121,20.589476,23.48827,23.33776,21.32195,20.25615,19.54544,8102,301,3,110,5.658977e+18,QSO,1.424659,5026,55855,741
6,1.237679e+18,23.234926,11.418188,21.46973,21.17624,20.92829,20.60826,20.42573,7773,301,2,462,1.246262e+19,QSO,0.586455,11069,58456,113
7,1.237679e+18,5.433176,12.065186,22.24979,22.02172,20.34126,19.48794,18.84999,7773,301,2,346,6.961443e+18,GALAXY,0.477009,6183,56210,15
8,1.237661e+18,200.290475,47.199402,24.40286,22.35669,20.61032,19.46490,18.95852,3716,301,5,108,7.459285e+18,GALAXY,0.660012,6625,56386,719
9,1.237671e+18,39.149691,28.102842,21.74669,20.03493,19.17553,18.81823,18.65422,5934,301,4,122,2.751763e+18,STAR,-0.000008,2444,54082,232


In [3]:
df_raw.dtypes

obj_ID         float64
alpha          float64
delta          float64
u              float64
g              float64
r              float64
i              float64
z              float64
run_ID           int64
rerun_ID         int64
cam_col          int64
field_ID         int64
spec_obj_ID    float64
class           object
redshift       float64
plate            int64
MJD              int64
fiber_ID         int64
dtype: object

In [4]:
# Remove the Object ID, Run ID, Rerun ID, Unique ID, and MJD Columns. ID and date columns were
    # determined to be irrelevant to catigorization of stellar objects.
df_mod = df_raw.drop(columns=["obj_ID","run_ID","rerun_ID","spec_obj_ID","MJD","cam_col","plate","field_ID","fiber_ID"])
df_mod.head(10)

,alpha,delta,u,g,r,i,z,class,redshift
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123
5,340.995121,20.589476,23.48827,23.33776,21.32195,20.25615,19.54544,QSO,1.424659
6,23.234926,11.418188,21.46973,21.17624,20.92829,20.60826,20.42573,QSO,0.586455
7,5.433176,12.065186,22.24979,22.02172,20.34126,19.48794,18.84999,GALAXY,0.477009
8,200.290475,47.199402,24.40286,22.35669,20.61032,19.46490,18.95852,GALAXY,0.660012
9,39.149691,28.102842,21.74669,20.03493,19.17553,18.81823,18.65422,STAR,-0.000008


In [5]:
df_mod = df_mod.sort_values(by='z')
df_mod = df_mod.iloc[1:]
df_mod

,alpha,delta,u,g,r,i,z,class,redshift
14498,84.230172,7.138590,14.15199,10.73097,9.82207,9.469903,9.612333,STAR,0.000051
4472,239.423127,28.548438,12.26240,10.51139,10.06854,13.417860,10.225510,STAR,0.000025
75784,239.473361,27.594369,12.10168,10.49820,10.11604,10.008650,10.441310,STAR,0.000008
75620,239.207155,27.898444,12.30349,10.67180,10.19460,10.055090,10.650560,STAR,0.000003
85285,164.321732,40.221137,15.44840,11.79892,10.86379,10.566470,10.778890,STAR,-0.000159
...,...,...,...,...,...,...,...,...,...
29689,146.345791,24.566331,18.50142,18.51986,20.75484,30.163590,27.673360,GALAXY,1.086811
85950,255.213997,28.549537,20.65719,19.14651,20.99419,29.889210,27.805190,GALAXY,0.144278
40201,36.908884,-1.982394,20.00673,20.29219,18.54149,17.813190,28.238290,GALAXY,0.593243
93321,16.455371,0.321688,18.64831,18.43714,17.49240,32.141470,28.790550,QSO,1.773661


In [6]:
df_mod.isnull().sum()

alpha       0
delta       0
u           0
g           0
r           0
i           0
z           0
class       0
redshift    0
dtype: int64

In [7]:
# Transform Class column
def changeStatus(stellar_class):
    if stellar_class == "GALAXY":
        return 0
    elif stellar_class == "STAR":
        return 1
    else:
        return -1
    
df_mod["class"] = df_mod["class"].apply(changeStatus)
df_mod.head(10)

,alpha,delta,u,g,r,i,z,class,redshift
14498,84.230172,7.138590,14.15199,10.73097,9.82207,9.469903,9.612333,1,0.000051
4472,239.423127,28.548438,12.26240,10.51139,10.06854,13.417860,10.225510,1,0.000025
75784,239.473361,27.594369,12.10168,10.49820,10.11604,10.008650,10.441310,1,0.000008
75620,239.207155,27.898444,12.30349,10.67180,10.19460,10.055090,10.650560,1,0.000003
85285,164.321732,40.221137,15.44840,11.79892,10.86379,10.566470,10.778890,1,-0.000159
81298,8.511630,-9.705282,14.56010,12.68849,11.77229,11.319370,10.897380,0,0.012450
4401,184.282050,7.624347,14.50678,12.67902,11.74664,11.299560,10.918470,0,0.007590
75583,239.702668,27.467439,12.99664,11.33897,10.98255,10.873740,11.194480,1,0.000174
57763,208.358688,33.825699,10.99623,13.66217,12.35763,12.637440,11.302470,-1,3.955592
76986,240.602807,26.945961,15.32899,11.47435,11.09069,10.956650,11.414840,1,0.000049


In [8]:
# Define the features set.
X = df_mod.copy()
X = X.drop(columns=["class"])

In [9]:
# Define the target set.
y = df_mod["class"]

In [10]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Neural Network Model

In [12]:
# Define the number of input features and hidden layers
number_input_features = 8
hidden_nodes_layer1 = [20,20,40,40,60,60,80,80,100,100]
hidden_nodes_layer2 = [5,9,5,9,5,9,5,9,5,9]

# Predefine arrays
model_loss = [0,0,0,0,0,0,0,0,0,0]
model_accuracy = [0,0,0,0,0,0,0,0,0,0]

In [13]:
# Creating multiple iterations of Neural Networks to determine 
counter = 0
for i in hidden_nodes_layer1:
    # Define the model - deep neural net
    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1[counter], input_dim=number_input_features, activation="relu"))

    # Second hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2[counter], activation="relu"))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    # Train the model
    fit_model = nn.fit(X_train_scaled,y_train,epochs=25)

    # Evaluate the model using the test data
    model_loss[counter], model_accuracy[counter] = nn.evaluate(X_test_scaled,y_test,verbose=2)
    
    print(f"\n Iteration {counter} completed. \n")
    counter = counter + 1

Epoch 1/25
2344/2344 [==============================] - 3s 936us/step - loss: -2.1140 - accuracy: 0.6046
Epoch 2/25
2344/2344 [==============================] - 2s 882us/step - loss: -2.4950 - accuracy: 0.6338
Epoch 3/25
2344/2344 [==============================] - 2s 871us/step - loss: -2.6041 - accuracy: 0.7055
Epoch 4/25
2344/2344 [==============================] - 2s 862us/step - loss: -2.6981 - accuracy: 0.7444
Epoch 5/25
2344/2344 [==============================] - 2s 861us/step - loss: -2.7603 - accuracy: 0.7623
Epoch 6/25
2344/2344 [==============================] - 2s 916us/step - loss: -2.7975 - accuracy: 0.7785
Epoch 7/25
2344/2344 [==============================] - 2s 1ms/step - loss: -2.8217 - accuracy: 0.7871
Epoch 8/25
2344/2344 [==============================] - 2s 882us/step - loss: -2.8381 - accuracy: 0.7912
Epoch 9/25
2344/2344 [==============================] - 2s 920us/step - loss: -2.8532 - accuracy: 0.7942
Epoch 10/25
2344/2344 [==============================] - 


 Iteration 2 completed. 

Epoch 1/25
2344/2344 [==============================] - 3s 958us/step - loss: -2.6703 - accuracy: 0.7456
Epoch 2/25
2344/2344 [==============================] - 2s 948us/step - loss: -2.7814 - accuracy: 0.7797
Epoch 3/25
2344/2344 [==============================] - 2s 881us/step - loss: -2.8197 - accuracy: 0.7870
Epoch 4/25
2344/2344 [==============================] - 2s 850us/step - loss: -2.8567 - accuracy: 0.7948
Epoch 5/25
2344/2344 [==============================] - 2s 816us/step - loss: -2.7387 - accuracy: 0.7858
Epoch 6/25
2344/2344 [==============================] - 2s 807us/step - loss: -2.8678 - accuracy: 0.7971
Epoch 7/25
2344/2344 [==============================] - 2s 806us/step - loss: -2.8359 - accuracy: 0.7921
Epoch 8/25
2344/2344 [==============================] - 2s 807us/step - loss: -2.8715 - accuracy: 0.7984
Epoch 9/25
2344/2344 [==============================] - 2s 811us/step - loss: -2.8686 - accuracy: 0.7976
Epoch 10/25
2344/2344 [=====


 Iteration 5 completed. 

Epoch 1/25
2344/2344 [==============================] - 2s 830us/step - loss: 0.4176 - accuracy: 0.5943
Epoch 2/25
2344/2344 [==============================] - 2s 792us/step - loss: 0.4173 - accuracy: 0.5943
Epoch 3/25
2344/2344 [==============================] - 2s 831us/step - loss: 0.4173 - accuracy: 0.5943
Epoch 4/25
2344/2344 [==============================] - 2s 827us/step - loss: 0.4173 - accuracy: 0.5943
Epoch 5/25
2344/2344 [==============================] - 2s 821us/step - loss: 0.4173 - accuracy: 0.5943
Epoch 6/25
2344/2344 [==============================] - 2s 823us/step - loss: 0.4173 - accuracy: 0.5943
Epoch 7/25
2344/2344 [==============================] - 2s 834us/step - loss: 0.4173 - accuracy: 0.5943
Epoch 8/25
2344/2344 [==============================] - 2s 824us/step - loss: 0.4173 - accuracy: 0.5943
Epoch 9/25
2344/2344 [==============================] - 2s 824us/step - loss: 0.4173 - accuracy: 0.5943
Epoch 10/25
2344/2344 [==============

2344/2344 [==============================] - 3s 963us/step - loss: -2.6652 - accuracy: 0.7395
Epoch 2/25
2344/2344 [==============================] - 2s 946us/step - loss: -2.7777 - accuracy: 0.7790
Epoch 3/25
2344/2344 [==============================] - 2s 946us/step - loss: -2.7816 - accuracy: 0.7807
Epoch 4/25
2344/2344 [==============================] - 2s 908us/step - loss: -2.8528 - accuracy: 0.7939
Epoch 5/25
2344/2344 [==============================] - 2s 903us/step - loss: -2.8561 - accuracy: 0.7938
Epoch 6/25
2344/2344 [==============================] - 2s 900us/step - loss: -2.8421 - accuracy: 0.7932
Epoch 7/25
2344/2344 [==============================] - 2s 902us/step - loss: -2.8465 - accuracy: 0.7934
Epoch 8/25
2344/2344 [==============================] - 2s 906us/step - loss: -2.8672 - accuracy: 0.7974
Epoch 9/25
2344/2344 [==============================] - 2s 905us/step - loss: -2.8334 - accuracy: 0.7939
Epoch 10/25
2344/2344 [==============================] - 2s 912us/

In [14]:
counter = 0
for j in model_accuracy:
    print(model_accuracy[counter])
    counter = counter + 1

0.800599992275238
0.7983199954032898
0.7979199886322021
0.799239993095398
0.8010799884796143
0.800000011920929
0.5950000286102295
0.8003600239753723
0.5950000286102295
0.7982800006866455
